In [100]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
import random
from math import sqrt
import pymongo
from pymongo import MongoClient
import json
import tensorflow as tf

In [2]:
connection = MongoClient('localhost', 27017)
db = connection.recsys

In [3]:
ratings = db.ratings
movies = db.movies
recommendations = db.recommendations

In [4]:
df = pd.DataFrame(list(ratings.find()))
movies = pd.DataFrame(list(movies.find()))

In [5]:
new_df = pd.concat([movies.drop('genres', 1), movies['genres'].str.get_dummies(sep="|")], 1)

In [6]:
new_df.drop(new_df[new_df['(no genres listed)']==1].index,inplace=True)

In [7]:
new_df = new_df.drop(['(no genres listed)'], axis = 1) 

In [8]:
df = pd.merge(df, new_df, on='movieId')

In [15]:
genre_cols = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 
       'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'IMAX', 
       'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']

In [35]:
NO_OF_GENRES = 19

In [154]:
def rgr_calculation(user):
    tr = tf.Variable(np.sum(user['rating']),dtype=tf.float32)
    rgr=tf.Variable([0]*NO_OF_GENRES,dtype=tf.float32)
    j=0
    for i in genre_cols:
        rgr[j].assign(user[(user[i]==1)&(user['rating']>=3)]['rating'].sum())
        rgr[j].assign(rgr[j]/tr)
        j=j+1
    rgr = tf.convert_to_tensor(rgr, dtype=tf.float32)
    return rgr

In [170]:
def mrgf_calculation(user):
    totalFrequency = tf.Variable(np.size(user['rating']),dtype=tf.float32)
    mrgf = tf.Variable([0]*NO_OF_GENRES,dtype = tf.float32)
    j = 0
    for i in genre_cols:
        o1 = tf.Variable(np.size(user[(user[i]==1)&(user['rating']==3)]['rating'].values),dtype = tf.float32)
        o2 = tf.Variable(np.size(user[(user[i]==1)&(user['rating']==4)]['rating'].values),dtype = tf.float32)
        o3 = tf.Variable(np.size(user[(user[i]==1)&(user['rating']==5)]['rating'].values),dtype = tf.float32)
        mrgf[j].----------------0assign(o1+2*o2+3*o3)
        mrgf[j].assign(mrgf[j]/(3*totalFrequency))
        j=j+1
        mrgf = tf.convert_to_tensor(mrgf, dtype=tf.float32)
    return mrgf

In [171]:
mrgf_calculation(df[df['userId']==1])

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

In [124]:
def gim_calculation(user):
    rgr=tf.Variable(dtype=tf.float32)
    rgr = rgr_calculation(user)
    mrgf = mrgf_calculation(user)
    gim_list = [0]*NO_OF_GENRES
    nf = 5.0
    for i in range(0, NO_OF_GENRES):
        if((rgr[i] + mrgf[i])==0):
            gim_list[i]=0
        else:
            gim_list[i] = (2 * nf * mrgf[i] * rgr[i]) / (rgr[i] + mrgf[i])
    gim_list = np.nan_to_num(gim_list)
    return gim_list

In [99]:
class GIM:
    """GIM- Genre Interestingness Measure"""

    def __init__(self):
        pass

    def gim_a(self, gim, i):
        """Method to get fuzzy set value for very_bad, bad, average, good."""
        if gim <= i - 2 or gim > i:
            return 0.0
        elif i - 2 < gim <= i - 1:
            return gim - i + 2.0
        elif i - 1 < gim <= i:
            return float(i - gim)

    def very_bad(self, gim):
        if gim <= 1.0:
            return 1.0
        else:
            return 0.0

    def bad(self, gim):
        return self.gim_a(gim, 2.0)

    def average(self, gim):
        return self.gim_a(gim, 3.0)

    def good(self, gim):
        return self.gim_a(gim, 4.0)

    def very_good(self, gim):
        return self.gim_a(gim, 5.0)

    def excellent(self, gim):
        if gim <=4.0:
            return 0.0
        else:
            return (gim-4.0)

    def get_fuzzy_set(self, gim_value):
        """Get fuzzy set of gim(list of values) based on given gim value."""
        return [self.very_bad(gim_value),
                self.bad(gim_value),
                self.average(gim_value),
                self.good(gim_value),
                self.very_good(gim_value),
                self.excellent(gim_value)]